Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Evaluation code


__Disclaimer__
*   This notebook contains experimental code, which may be changed without notice.
*   The ideas here are some ideas relevant to fairness - they are not the whole story!



# Notebook summary

This notebook intends to evaluate a list of models on two dimensions:
- "Performance": How well the model perform to classify the data (intended bias). Currently, we use the AUC.
- "Bias": How much bias does the model contain (unintended bias). Currently, we use the pinned auc.

This script takes the following steps:

- Prepare the data:
    - a "performance dataset" which will be used for the first set of metrics. This dataset is supposed to be similar format to the training data (contain a text and a label).
    - a "bias dataset" which will be used for the second set of metrics. This data contains a text, a label but also some subgroup information to evaluate the unintended bias on.
- Runs predictions: we will convert both datasets to TF-Records and call a batch prediction job on Cloud MLE. The result will be added to our data.
- Evaluate metrics.

# Settings

We start by loading some libraries that we will use and customizing the visualization parameters.

In [ ]:
!pip install -U -q git+https://github.com/conversationai/unintended-ml-bias-analysis@1de676a31de9e43892964f71d1e38e90fc8b331e

In [ ]:
from unintended_ml_bias import model_bias_analysis

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import getpass
import json
import numpy as np
import pandas as pd
import pkg_resources
import os
import re
import seaborn as sns
import time

import googleapiclient.discovery as discovery
import googleapiclient.errors as errors
import tensorflow as tf
from tensorflow.python.lib.io import file_io

In [ ]:
cm = sns.light_palette("red", as_cmap=True)

In [ ]:
os.environ['GCS_READ_CACHE_MAX_SIZE_MB'] = '0' #Faster to access GCS file + https://github.com/tensorflow/tensorflow/issues/15530

#### Setting project config

In [ ]:
# User inputs.
PROJECT_NAME = 'wikidetox'

# Dataset preparation

#### Requirements: 

Cleaned datasets must be pandas DataFrames and must include the following fields:
- `text`: the raw text string of the comment.
- `label`: label associated to this comment. Must be True for toxic.

### Preparing performance set

In [ ]:
# User inputs.
PERFORMANCE_DATASET = 'gs://kaggle-model-experiments/resources/toxicity_q42017_test.tfrecord'
SAMPLE_SIZE_PERFORMANCE = 5000 # Set to None to use all data.

TEXT_FEATURE_NAME = 'comment_text'
SENTENCE_KEY = 'comment_key'
LABEL_NAME = 'frac_neg'

In [ ]:
def load_tf_records_to_pandas(tf_records_path, text_feature_name, label_name, max_n_records=None):
  '''Loads tf-records into a pandas dataframe.'''
    
  if not max_n_records:
    max_n_records = float('inf')
    
  # Read TFRecord file
  reader = tf.TFRecordReader()
  filename_queue = tf.train.string_input_producer([tf_records_path], num_epochs=1)

  _, serialized_example = reader.read(filename_queue)

  # Define features
  read_features = {
      text_feature_name: tf.FixedLenFeature([], dtype=tf.string),
      label_name: tf.FixedLenFeature([], dtype=tf.float32)
  }

  # Extract features from serialized data
  read_data = tf.parse_single_example(serialized=serialized_example,
                                      features=read_features)
  
  # Read and print data:
  sess = tf.InteractiveSession()
  
  # Many tf.train functions use tf.train.QueueRunner,
  # so we need to start it before we read.
  sess.run(tf.global_variables_initializer())
  sess.run(tf.local_variables_initializer())
  sess.run(tf.tables_initializer())
  tf.train.start_queue_runners(sess)
  
  d = []
  new_line = sess.run(read_data)
  count = 0
  while new_line:
    d.append(new_line)
    count += 1
    if count >= max_n_records:
      break
    try:
      new_line = sess.run(read_data)
    except tf.errors.OutOfRangeError:
      print ('End of file.')
      break
    if not(count % 100000):
      print ('Loaded {} lines.'.format(count))

  return pd.DataFrame(d)

In [ ]:
test_performance_df = load_tf_records_to_pandas(
    PERFORMANCE_DATASET,
    TEXT_FEATURE_NAME,
    LABEL_NAME,
    max_n_records=SAMPLE_SIZE_PERFORMANCE,
    )

In [ ]:
# Setting the table to match the required format.
test_performance_df = test_performance_df.rename(
    columns={
        TEXT_FEATURE_NAME: 'text',
        LABEL_NAME: 'label'
    })
test_performance_df['label'] = list(map(lambda x :bool(round(x)), list(test_performance_df['label'])))

In [ ]:
print (len(test_performance_df))
test_performance_df.head()

### Preparing bias set

In [ ]:
# User inputs.
BIAS_SAMPLE_SIZE = 5000 # Set to None to use all data.

In [ ]:
# Loading it from it the unintended_ml_bias github.
test_bias_df = pd.read_csv(
    pkg_resources.resource_stream("unintended_ml_bias", "eval_datasets/bias_madlibs_77k.csv"))
test_bias_df['text'] = test_bias_df['Text']
test_bias_df['label'] = test_bias_df['Label']
test_bias_df['label'] = list(map(lambda x: x=='BAD', test_bias_df['label']))
test_bias_df = test_bias_df[['text', 'label']].copy()
terms = [line.strip()
         for line in pkg_resources.resource_stream("unintended_ml_bias", "bias_madlibs_data/adjectives_people.txt")]
model_bias_analysis.add_subgroup_columns_from_text(test_bias_df, 'text', terms)

In [ ]:
if BIAS_SAMPLE_SIZE:
    test_bias_df = test_bias_df.sample(n=BIAS_SAMPLE_SIZE, random_state=2018)
    test_bias_df = test_bias_df.copy()
test_bias_df.head()

# Calling model to make predictions

##### Model families lists all the models to evaluate.

"Model Families" allows the results to capture training variance by grouping different training versions of each model together. model_families is a list of lists, each sub-list ("model_family") contains the names of different training versions of the same model.

##### Format.
MODEL_FAMILIES lists all the subfamilies. One subfamily is a list of models with the pattern (\$MODEL_NAME((:\$VERSION_NAME)?)).
If the version is not specified, the default one is used.

In [ ]:
# User inputs.
MODEL_FAMILIES = [
    ['tf_gru_attention:v_20180914_163804']
    ]

### Utility functions

#### Converting dataframe to TF-Records.

In [ ]:
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _write_pandas_to_tf_records(df, gcs_path):
  '''Write a pandas `DataFrame` to a tf_record.
  
  Args:
    df: pandas `DataFrame`. It must include the fields 'sentence'.
    gcs_path: where to write the tf records.
  
  Note: TFRecords will have fields `sentence` and `key`.
  '''
  
  writer = tf.python_io.TFRecordWriter(gcs_path)
  for i in range(len(df)):
    
      if not i % 10000:
          print ('Preparing train data: {}/{}'.format(i, len(df)))
      
      # Create a feature
      feature = {TEXT_FEATURE_NAME: _bytes_feature(tf.compat.as_bytes(df['text'].iloc[i])),
                 SENTENCE_KEY: _int64_feature(i)}
      example = tf.train.Example(features=tf.train.Features(feature=feature))


      # Serialize to string and write on the file
      writer.write(example.SerializeToString())

  writer.close()

#### Running batch job.

In [ ]:
def _make_batch_job_body(project_name, input_paths, output_path,
        model_name, region='us-central1', data_format='TF_RECORD',
        version_name=None, max_worker_count=None,
        runtime_version=None):
  '''Creates the request body for Cloud MLE batch prediction job.'''

  project_id = 'projects/{}'.format(project_name)
  model_id = '{}/models/{}'.format(project_id, model_name)
  if version_name:
    version_id = '{}/versions/{}'.format(model_id, version_name)

  # Make a jobName of the format "model_name_batch_predict_YYYYMMDD_HHMMSS"
  timestamp = time.strftime('%Y%m%d_%H%M%S', time.gmtime())

  # Make sure the project name is formatted correctly to work as the basis
  # of a valid job name.
  clean_project_name = re.sub(r'\W+', '_', project_name)

  job_id = '{}_{}_{}'.format(clean_project_name, model_name,
                             timestamp)

  # Start building the request dictionary with required information.
  body = {'jobId': job_id,
          'predictionInput': {
              'dataFormat': data_format,
              'inputPaths': input_paths,
              'outputPath': output_path,
              'region': region
          }}

  # Use the version if present, the model (its default version) if not.
  if version_name:
    body['predictionInput']['versionName'] = version_id
  else:
    body['predictionInput']['modelName'] = model_id

  # Only include a maximum number of workers or a runtime version if specified.
  # Otherwise let the service use its defaults.
  if max_worker_count:
    body['predictionInput']['maxWorkerCount'] = max_worker_count

  if runtime_version:
    body['predictionInput']['runtimeVersion'] = runtime_version

  return body


def _call_batch_job(project_name, input_paths, output_path, model_name, version_name=None):
  '''Calls a batch prediction job on Cloud MLE.'''
  
  batch_predict_body = _make_batch_job_body(
      project_name, input_paths, output_path, model_name, version_name=version_name)

  project_id = 'projects/{}'.format(project_name)

  ml = discovery.build('ml', 'v1')
  request = ml.projects().jobs().create(parent=project_id,
                                        body=batch_predict_body)

  try:
    response = request.execute()
    print('state : {}'.format(response['state']))
    return response['jobId']

  except errors.HttpError as err:
    # Something went wrong, print out some information.
    print('There was an error getting the prediction results.' +
          'Check the details:')
    print(err._get_reason())

#### Map predictions results to df

In [ ]:
def _check_job_over(project_name, job_name):
  '''Sleeps until the batch job is over.'''
  
  project_id = 'projects/{}'.format(project_name)
  clean_project_name = re.sub(r'\W+', '_', project_name)
  
  ml = discovery.build('ml', 'v1')
  request = ml.projects().jobs().get(name='projects/{}/jobs/{}'.format(clean_project_name, job_name))
  
  job_completed = False
  k = 0
  while not job_completed:
    k += 1
    response = request.execute()
    job_completed = (response['state'] == 'SUCCEEDED')
    if not (k % 5) and not job_completed:
      print ('Waiting for prediction job to complete. Min elapsed: {}'.format(0.5*k))
      time.sleep(30)
  
  print ('Prediction job completed.')

    
def _combine_prediction_to_df(df, prediction_file, model_col_name):
  '''Loads the prediction files and adds them to the DataFrame.'''
  
  def load_predictions(prediction_file):
    with file_io.FileIO(prediction_file, 'r') as _file:
      # prediction file needs to fit in memory.
      predictions = [json.loads(line) for line in _file] 
    return predictions
  
  predictions = load_predictions(prediction_file)
  predictions = sorted(predictions, key = lambda x: x[SENTENCE_KEY])
  
  if len(predictions) != len(df):
    raise ValueError('The dataframe and the prediction file do not contain the same number of lines.')
  
  prediction_proba = [x[LABEL_NAME][0] for x in predictions]
  
  df[model_col_name] = prediction_proba
  
  return df

#### Combine everything in one single function

In [ ]:
# High level function that should be used by user.

def call_model_predictions_from_df(df, project_name, tmp_tfrecords_gcs_path, 
                                   tmp_tfrecords_with_predictions_gcs_path, model_name, 
                                   version_name=None, rewrite=True):
  '''Calls a prediction job.
  
  Args:
    - df: a pandas `DataFrame`. Must contain a `text` field.
    - project_name: gcp project name.
    - tmp_tfrecords_gcs_path: gcs path to store tf_records, which will be inputs
        to batch prediction job.
    - tmp_tfrecords_with_predictions_gcs_path: gcs path to store tf_records, 
        which will be outputs to batch prediction job.
    - model_name: Model name used to run predictions.
        The model must take as inputs TF-Records with fields $TEXT_FEATURE_NAME
        and $SENTENCE_KEY, and should return a dictionary including the field $LABEL_NAME.
    - version_name: Model version to run predictions.
        If None, it will use default version of the model.
    - rewrite: whether to rewrite the tmp_tfrecords_gcs_path.
        If False, it will check if file is already existing and will potentially
        use pre-existing file to call predictions, without re-running preprocessing.

  Returns:
    - job_id: the job_id of the prediction job.
  '''
  
  # Create tf-records if necessary.
  if rewrite or not file_io.file_exists(tmp_tfrecords_gcs_path):
    _write_pandas_to_tf_records(df, tmp_tfrecords_gcs_path)
  
  # Call batch prediction job. 
  job_id = _call_batch_job(
    project_name,
    input_paths=tmp_tfrecords_gcs_path,
    output_path=tmp_tfrecords_with_predictions_gcs_path,
    model_name=model_name,
    version_name=version_name)
  
  return job_id


def add_model_predictions_to_df(job_id, df, project_name,
                                tmp_tfrecords_with_predictions_gcs_path, column_name_of_model):
  '''Adds prediction results to the pandas dataframe.
  
  Args:
    - job_id: the job_id of the prediction job.
    - df: a pandas `DataFrame`. Must contain a `text` field.
    - project_name: gcp project name.
    - tmp_tfrecords_with_predictions_gcs_path: gcs path to store tf_records, 
        which will be outputs to batch prediction job.
    - column_name_of_model: Name of the added column.
  
  Returns:
    - df: a pandas ` DataFrame` with an added column named 'column_name_of_model'
        containing the prediction values.
  '''

  # Waits for batch job to be over.
  _check_job_over(project_name, job_id)
  
  # Add one prediction column to the database.
  tf_records_path = os.path.join(tmp_tfrecords_with_predictions_gcs_path, 'prediction.results-00000-of-00001')
  df_with_predictions = _combine_prediction_to_df(df, tf_records_path, column_name_of_model)
  
  return df_with_predictions

### Running entire pipeline

In [ ]:
# User inputs.
GCS_BUCKET='gs://kaggle-model-experiments/'

TF_RECORD_PERFORMANCE_INPUT = os.path.join(
    GCS_BUCKET,
    getpass.getuser(),
    'tfrecords/test_performance.tfrecords')
TF_RECORD_PERFORMANCE_PREDICTION_PREFIX = os.path.join(
    GCS_BUCKET,
    getpass.getuser(),
    'tfrecords/test_performance_with_predictions_') 

TF_RECORD_BIAS_INPUT = os.path.join(
    GCS_BUCKET,
    getpass.getuser(),
    'test_bias.tfrecords'
)
TF_RECORD_BIAS_PREDICTION_PREFIX = os.path.join(
    GCS_BUCKET,
    getpass.getuser(),
    'test_bias_with_predictions_'
)

In [ ]:
# Running predictions
job_ids_performance = []
job_ids_bias = []
for subfamily in MODEL_FAMILIES:
  for model_full_name in subfamily:
        
    model_full_name_split = model_full_name.split(':')
    model = model_full_name_split[0]
    if len(model_full_name_split) > 1:
      version = model_full_name_split[1]
    else:
      version = None
        
    job_id = call_model_predictions_from_df(
        test_performance_df,
        project_name=PROJECT_NAME,
        tmp_tfrecords_gcs_path=TF_RECORD_PERFORMANCE_INPUT,
        tmp_tfrecords_with_predictions_gcs_path=TF_RECORD_PERFORMANCE_PREDICTION_PREFIX + model_full_name,
        model_name=model,
        version_name=version
    )
    job_ids_performance.append(job_id)
        
    job_id = call_model_predictions_from_df(
        test_bias_df,
        project_name=PROJECT_NAME,
        tmp_tfrecords_gcs_path=TF_RECORD_BIAS_INPUT,
        tmp_tfrecords_with_predictions_gcs_path=TF_RECORD_BIAS_PREDICTION_PREFIX+ model_full_name,
        model_name=model,
        version_name=version
    )
    job_ids_bias.append(job_id)


# Collecting predictions        
i = 0 
for subfamily in MODEL_FAMILIES:
  for model_full_name in subfamily:
    
    job_id = job_ids_performance[i]
    test_performance_df = add_model_predictions_to_df(
        job_id,
        test_performance_df,
        project_name=PROJECT_NAME,
        tmp_tfrecords_with_predictions_gcs_path=TF_RECORD_BIAS_PREDICTION_PREFIX + model_full_name,
        column_name_of_model=model_full_name
    )
        
    job_id = job_ids_bias[i]
    test_bias_df = add_model_predictions_to_df(
        job_id,
        test_bias_df,
        project_name=PROJECT_NAME,
        tmp_tfrecords_with_predictions_gcs_path=TF_RECORD_BIAS_PREDICTION_PREFIX + model_full_name,
        column_name_of_model=model_full_name
    )
    i += 1

In [ ]:
test_performance_df.head()

In [ ]:
test_bias_df.head()

# Run evaluation metrics

## Performance metrics

### Data Format

At this point, our performance data is in DataFrame df, with columns:

text: Full text of the comment.
label: True if the comment is Toxic, False otherwise.
< model name >: One column per model, cells contain the score from that model.
You can run the analysis below on any data in this format. Subgroup labels can be generated via words in the text as done above, or come from human labels if you have them.

### Run AUC

In [ ]:
import sklearn.metrics as metrics

In [ ]:
for model_family in MODEL_FAMILIES:
  auc_list = []
  for model in model_family:
    fpr, tpr, thresholds = metrics.roc_curve(test_performance_df['label'], test_performance_df[model])
    auc_list.append(metrics.auc(fpr, tpr))
  print ('Auc for model {}: {}'.format(model, np.mean(auc_list)))

## Unintended Bias Metrics

### Data Format
At this point, our bias data is in DataFrame df, with columns:

*   text: Full text of the comment.
*   label: True if the comment is Toxic, False otherwise.
*   < model name >: One column per model, cells contain the score from that model.
*   < subgroup >: One column per identity, True if the comment mentions this identity.

You can run the analysis below on any data in this format. Subgroup labels can be 
generated via words in the text as done above, or come from human labels if you have them.


### Pinned AUC
Pinned AUC measures the extent of unintended bias of a real-value score function
by measuring each sub-group's divergence from the general distribution.

Let $D$ represent the full data set and $D_g$ be the set of examples in subgroup
$g$. Then:


$$ Pinned \ dataset \ for \ group \ g = pD_g = s(D_g) + s(D), |s(D_g)| = |s(D)| $$

$$ Pinned \ AUC \ for \ group \ g = pAUC_g = AUC(pD_g) $$

$$ Pinned \ AUC \ Squared \ Equality \ Difference = \Sigma_{g \in G}(AUC - pAUC_g)^2 $$


### Pinned AUC Equality Difference
The table below shows the pinned AUC equality difference for each model family.
Lower scores (lighter red) represent more similarity between each group's pinned AUC, which means
less unintended bias.

On this set, the wiki_debias_cnn model demonstrates least unintended bias. 

In [ ]:
eq_diff = model_bias_analysis.per_subgroup_auc_diff_from_overall(
    test_bias_df, terms, MODEL_FAMILIES, squared_error=True, normed_auc=True)
# sort to guarantee deterministic output
eq_diff.sort_values(by=['model_family'], inplace=True)
eq_diff.reset_index(drop=True, inplace=True)
eq_diff.style.background_gradient(cmap=cm)

### Pinned AUC Graphs
The graphs below show per-group Pinned AUC for each subgroup and each model. Each
identity group shows 3 points, each representing the pinned AUC for one training 
version of the model. More consistency among the values represents less unintended bias.

In [ ]:
pinned_auc_results = model_bias_analysis.per_subgroup_aucs(test_bias_df, terms, MODEL_FAMILIES, 'label')
for family in MODEL_FAMILIES:
  name = model_bias_analysis.model_family_name(family)
  model_bias_analysis.per_subgroup_scatterplots(
      pinned_auc_results,
      'subgroup',
      name + '_aucs',
      name + ' Pinned AUC',
      y_lim=(0., 1.0))